## Creating a convolutional neural network (CNN)

In this notebook, we will import the information stored as numpy arrays and create train/test and validation data sets. As we have imbalanced data due to the high abundance of beech trees in the data set, we will do data augmentation and oversampling. Next, we crate the structure for the models used for classification and look into the results and accuracy of our best model.

While we tested different model structures and number of features, here, we will only report our best model.


**Data you need for running this notebook:**

Zipped numpy arrays (.npz) with all information, that should be added to the model (such as spectral information and vegetation height)

**Content overview:**

- Setup
- Import and manipulate data
- Split the data into a train, test and validation set
- Perform data augmentation
- Deal with class imbalance
- Perform oversampling
- One hot encoding
- Hyperparameter search
- Structure of the nsemble Model
- Results


---------

## Setup

In [1]:
# Toolboxes for data handling
import numpy as np
import random

# Toolboxes for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Toolboxes for modelling with a neural network
import tensorflow as tf
import mlflow

# Toolboxes for splitting the data
from sklearn.model_selection import train_test_split

#Toolbox for hyperparameter tuning
import kerastuner as kt

/var/folders/9g/jlj5g_zj4rg83ktdjhr98bf80000gn/T/ipykernel_26384/346208476.py:17: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


## Import and manipulate data 

Basis for the modelling are the box images, that have been stored as zipped numpy arrays (from notebook XXX)

In [2]:
# Define the path to the data file
data_path = '../images/box_images/baseline/model_images.npz'
height_path = '../images/box_images/baseline/model_images_vegheight.npz'

# Load the image array containing all box images in a 4D array and their
# labels
with np.load(data_path, allow_pickle=True) as data:
    img_array = data["img_array"]
    labels = data["labels"]

with np.load(height_path, allow_pickle=True) as data:
    height_array_tmp = data["img_array"]
    labels_height = data["labels"]

# Extract only none nan values from the year 2022
img_array = img_array[:,0:1701,:,:,:]
img_array = img_array[[4,4,4,4,4],:,:,:,:] ## 4 refers to 2022
labels = labels[:,0:1701]

height_array_tmp = height_array_tmp[0:1701,:,:]
height_array = np.full((5,1701,11,11), fill_value=np.nan)
height_array[0,:,:,:] = height_array_tmp
height_array[1,:,:,:] = height_array_tmp
height_array[2,:,:,:] = height_array_tmp
height_array[3,:,:,:] = height_array_tmp
height_array[4,:,:,:] = height_array_tmp

max_height = np.max(height_array_tmp)
min_height = np.min(height_array_tmp)

FileNotFoundError: [Errno 2] No such file or directory: '../images/box_images/baseline/model_images.npz'

### Assign labels as integers to different trees species

As tensorflow prefers integers compared to strings as target values, we transform our labels array. As a first step, we create a new array that looks exactly like labels but is filled with zeros

*Kommentar Caro:*
Hier werden nur die eine Ahorn und die eine Birkenart codiert, die anderen beiden Arten der jeweiligen Gattungen fallen damit noch in die 'Other' Kategorie.

Nochmal diskutieren/anpassen!

In [ ]:
# New array filled with 0s
labels_int = np.zeros_like(labels)

# Loop through each label
for i in range(len(labels[1])):
    
    # Assign a 1 if it is a Rotbuche
    if labels[1][i] == 'Rotbuche':
        labels_int[:,i] = 1

    # Assign a 2 if it is a Berg-Ahorn    
    elif labels[1][i] == 'Berg-Ahorn':
        labels_int[:,i] = 2

    # Assign a 3 if it is a Haenge-Birke    
    elif labels[1][i] == 'Haenge-Birke':
        labels_int[:,i] = 3
    
# As the array is still of type "obj", we need to change it to "int"
labels_int = labels_int.astype('int')

# # Only use the classes that are really interesting for us
# img_array = img_array[:,(labels_int[1] != 0),:,:,:]
# labels_int = labels_int[:,(labels_int[1] != 0)] - 1

## Split the data into a train, test and validation set

In [ ]:
# Create as many indices as there are trees (irrespective of years)
ind = np.arange(0, img_array.shape[1], 1)

# Split the indices into a train and temporary test set
ind_train, ind_test_tmp, label_train, label_test_tmp = \
    train_test_split(ind, labels_int[1], train_size=0.8,
                     stratify=labels_int[1], random_state=42)

# Split the temporary test indices once again into a test and validation set
ind_test, ind_validation, label_test, label_validation = \
    train_test_split(ind_test_tmp, label_test_tmp, train_size=0.5, 
                     stratify=label_test_tmp, random_state=42)

# Extract images and labels for the train data set using the train indices
# and reshape them so that there is once again only one dimension of images
# irrespective of the year they are coming from
img_train = img_array[:,ind_train,:,:,:].\
    reshape(labels_int.shape[0]*ind_train.shape[0],35,35,4)
height_train = height_array[:,ind_train,:,:].\
    reshape(labels_int.shape[0]*ind_train.shape[0],11,11)
label_train = labels_int[:,ind_train].\
    reshape(labels_int.shape[0]*ind_train.shape[0])

# Extract images and labels for the test data set using the test indices
# and reshape them so that there is once again only one dimension of images
# irrespective of the year they are coming from
img_test = img_array[:,ind_test,:,:,:].\
    reshape(labels_int.shape[0]*ind_test.shape[0],35,35,4)
height_test = height_array[:,ind_test,:,:].\
    reshape(labels_int.shape[0]*ind_test.shape[0],11,11)
label_test = labels_int[:,ind_test].\
    reshape(labels_int.shape[0]*ind_test.shape[0])

# Extract images and labels for the validation data set using the validation
# indices and reshape them so that there is once again only one dimension of
# images irrespective of the year they are coming from
img_validation = img_array[:,ind_validation,:,:,:].\
    reshape(labels_int.shape[0]*ind_validation.shape[0],35,35,4)
height_validation = height_array[:,ind_validation,:,:].\
    reshape(labels_int.shape[0]*ind_validation.shape[0],11,11)    
label_validation = labels_int[:,ind_validation].\
    reshape(labels_int.shape[0]*ind_validation.shape[0])



## Perform data augmentation for a given set of images


To deal with the low sample size of the data set, we will first create new images, by augmenting the existing ones.

We only perform flips and 90° rotations as other augmentations may interfere with vital information in the image - let's  be crazy and flip and rotate all images in all possible combinations (x12).

We first create a function for this:

In [ ]:
# Define a function that takes a dataset of images and augments each image
# once in a certain way
def augment_images(img_array):

    # Predefine an array that will contain the augmented images
    img_array_full = np.tile(img_array, (12,1,1,1,1))

    # Loop through each image
    for i in range(img_array.shape[0]):

        # Augmentation number 0: Nothing
        img_aug = img_array_full[0,i,:,:,:]
        img_array_full[0,i,:,:,:] = img_aug

        # Augmentation number 1: 90° rotation        
        img_aug = tf.image.rot90(img_array[i], k=1).numpy()
        img_array_full[1,i,:,:,:] = img_aug

        # Augmentation number 2: 90° rotation + vertical flip
        img_aug = tf.image.rot90(img_array[i], k=1).numpy()
        img_aug = tf.image.flip_up_down(img_aug).numpy()
        img_array_full[2,i,:,:,:] = img_aug

        # Augmentation number 3: 90° rotation + horizontal flip
        img_aug = tf.image.rot90(img_array[i], k=1).numpy()
        img_aug = tf.image.flip_left_right(img_aug).numpy()
        img_array_full[3,i,:,:,:] = img_aug

        # Augmentation number 4: 90° rotation + vertical flip +
        # horizontal flip
        img_aug = tf.image.rot90(img_array[i], k=1).numpy()
        img_aug = tf.image.flip_up_down(img_aug).numpy()
        img_aug = tf.image.flip_left_right(img_aug).numpy()
        img_array_full[4,i,:,:,:] = img_aug

        # Augmentation number 5: 270° rotation
        img_aug = tf.image.rot90(img_array[i], k=3).numpy()
        img_array_full[5,i,:,:,:] = img_aug

        # Augmentation number 6: 270° rotation + vertical flip
        img_aug = tf.image.rot90(img_array[i], k=3).numpy()
        img_aug = tf.image.flip_up_down(img_aug).numpy()
        img_array_full[6,i,:,:,:] = img_aug

        # Augmentation number 7: 90° rotation + horizontal flip
        img_aug = tf.image.rot90(img_array[i], k=3).numpy()
        img_aug = tf.image.flip_left_right(img_aug).numpy()
        img_array_full[7,i,:,:,:] = img_aug

        # Augmentation number 8: 270° rotation + vertical flip +
        # horizontal flip
        img_aug = tf.image.rot90(img_array[i], k=3).numpy()
        img_aug = tf.image.flip_up_down(img_aug).numpy()
        img_aug = tf.image.flip_left_right(img_aug).numpy()
        img_array_full[8,i,:,:,:] = img_aug

        # Augmentation number 9: vertical flip + horizontal flip
        img_aug = tf.image.flip_up_down(img_array[i]).numpy()
        img_aug = tf.image.flip_left_right(img_aug).numpy()
        img_array_full[9,i,:,:,:] = img_aug

        # Augmentation number 10: vertical flip
        img_aug = tf.image.flip_up_down(img_array[i]).numpy()
        img_array_full[10,i,:,:,:] = img_aug

        # Augmentation number 11: horizontal flip
        img_aug = tf.image.flip_left_right(img_array[i]).numpy()
        img_array_full[11,i,:,:,:] = img_aug

        # Report progress
        if (i%1000 == 0) & (i != 0):
            print(' - ' + str(round(100*i/img_array.shape[0])) + 
            '% of all images have been augmented.')
        elif (i%100 == 0) & (i != 0):
            print('.', end='')
        
    # Return the augmented dataset
    return img_array_full


Let´s use the function for augmentation:

In [ ]:
# Augment data
print('Augment spectral training images')
img_train_full = augment_images(img_train)
print('\n\nAugment vegetation height training images')
height_train_full = np.squeeze(augment_images(height_train[:,:,:,np.newaxis]))
print('\n\n"Augment" labels')
label_train_full = np.tile(label_train, (12,1))

# Reshape augmented data
img_train = np.reshape(img_train_full, (-1,35,35,4))
height_train = np.reshape(height_train_full, (-1,11,11))
label_train = np.reshape(label_train_full, (-1))

Augment spectral training images


2023-08-11 15:04:11.171384: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-08-11 15:04:11.171475: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-08-11 15:04:11.171489: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-08-11 15:04:11.172793: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-11 15:04:11.175516: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


......... - 15% of all images have been augmented.
......... - 29% of all images have been augmented.
......... - 44% of all images have been augmented.
......... - 59% of all images have been augmented.
......... - 74% of all images have been augmented.
......... - 88% of all images have been augmented.
.......

Augment vegetation height training images
......... - 15% of all images have been augmented.
......... - 29% of all images have been augmented.
......... - 44% of all images have been augmented.
......... - 59% of all images have been augmented.
......... - 74% of all images have been augmented.
......... - 88% of all images have been augmented.
.......

"Augment" labels


In [ ]:
# print(label_train.shape[0]/12)

# print(label_train[ 0*6800+1111])
# print(label_train[ 1*6800+1111])
# print(label_train[ 2*6800+1111])
# print(label_train[ 3*6800+1111])
# print(label_train[ 4*6800+1111])
# print(label_train[ 5*6800+1111])
# print(label_train[ 6*6800+1111])
# print(label_train[ 7*6800+1111])
# print(label_train[ 8*6800+1111])
# print(label_train[ 9*6800+1111])
# print(label_train[10*6800+1111])
# print(label_train[11*6800+1111])

Let´s check the shape of our train-datasets:

In [ ]:
print(img_train.shape)
print(height_train.shape)
print(label_train.shape)

(81600, 35, 35, 4)
(81600, 11, 11)
(81600,)


## Deal with class imbalance

As we have highly imbalanced data with 55% beech trees dominating the dataset, we explored different approaches to deal with imbalanced data. We tested and compared
- weighted classes
- oversampling
- undersampling

but found that **oversampling** worked best for our data.

### Perform oversampling

Here, we will randomly draw from the pool of original and augemented images to increase the sample sizes of the minorty classes, until the same size as for the majority class is reached.

In [ ]:
# Calculate the frequency of each class in the data
_, n_all = np.unique(label_train, return_counts=True)

# Determine the class with the most data
n_max = np.max(n_all)

# Generate a new array that contains the uosampled image results
img_os = img_train.copy()

# Generate a new array that contains the uosampled image results
height_os = height_train.copy()

# Generate a new array that contains the uosampled label results
label_os = label_train.copy()

# Loop through all class categories
for i in range(len(n_all)):

    # Extract the frequency of this class
    n_class = n_all[i]

    # Execute this code if this is not the class with the highest
    # frequency
    if (n_class != n_max):

        # Extract only images of this class
        img_class = img_train[label_train == i]

        # Extract only heights of this class
        height_class = height_train[label_train == i]        

        # Extract only labels of this class
        label_class = label_train[label_train == i]

        # Generate as many random integers as there are in the class
        # category with the highest frequency minus the number of already
        # existing images for this class category
        rand_ind = np.random.randint(0, img_class.shape[0]-1,
                                     n_max - n_class)

        # Draw random images from the existing images
        img_rand = img_class[rand_ind,:,:,:]

        # Draw random heights from the existing images
        height_rand = height_class[rand_ind,:,:]

        # Draw the very same random labels
        label_rand = label_class[rand_ind]

        # Append images to the image array
        img_os = np.append(img_os, img_rand, axis=0)

        # Append heights to the image array
        height_os = np.append(height_os, height_rand, axis=0)     

        # Append the very same random labels to the labels array
        label_os = np.append(label_os, label_rand, axis=0)



In [ ]:
label_os.shape

(181440,)

## One hot encoding

In [ ]:
# One hot encoding for the lables
label_os_ohe = tf.keras.utils.to_categorical(label_os)
label_test_ohe = tf.keras.utils.to_categorical(label_test)
label_validation_ohe = tf.keras.utils.to_categorical(label_validation)

## Hyper parameter search for a CNN

As the spectral information is the main part of our model, we will optimize the hyper parameters for this. We will use hyperparameter tuning to find a good kernel_initializer, the best learning rate as well as the best filter and dropuout rates.

We are the using Hyperband tuning algorithm, which uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model.




In [ ]:
#!pip install keras-tuner
from kerastuner import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

# Create a callback to stop training early after reaching a certain value for the validation loss.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

We have identified a good CNN structure, so we will use this here. 
As the Keras Tuner cannot work with Pooling layers, so they are not included.

In [ ]:
# Create the  model function

def model_builder(hp):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.InputLayer(input_shape=[35,35,4]))
    model.add(tf.keras.layers.Rescaling(1/((2**16)-1)))


    hp_filters1 = hp.Choice('filters1', values=[16, 32, 64]) #32 is best
    hp_dropout1 = hp.Choice('dropout1', values = [0.0, 0.2, 0.4])
    hp_initializer1 = hp.Choice('initializer1', values = ['uniform', 'he_normal','random_normal', 'he_uniform'])
    model.add(tf.keras.layers.Conv2D(filters=hp_filters1, kernel_size=[3, 3], strides=2, kernel_initializer= hp_initializer1, activation='relu'))
    model.add(tf.keras.layers.Dropout(hp_dropout1))

    hp_filters2 = hp.Choice('filters2', values=[64, 128, 256]) #128 is best
    hp_dropout2 = hp.Choice('dropout2', values = [0.3, 0.4, 0.5])
    model.add(tf.keras.layers.Conv2D(filters=hp_filters2, kernel_size=[3, 3], strides=2, kernel_initializer='uniform', activation='relu'))
    model.add(tf.keras.layers.Dropout(hp_dropout2))

    hp_filters3 = hp.Choice('filters3', values=[64, 128, 256]) #128 is best
    hp_dropout3 = hp.Choice('dropout3', values = [ 0.0, 0.2, 0.4])
    model.add(tf.keras.layers.Conv2D(filters=hp_filters3, kernel_size=[3,3], activation='relu'))
    model.add(tf.keras.layers.Dropout(hp_dropout3))


    model.add(tf.keras.layers.Flatten())

    hp_filters4 = hp.Choice('filters4', values=[16, 32, 64])
    hp_dropout4 = hp.Choice('dropout4', values = [ 0.0, 0.2, 0.4])
    model.add(tf.keras.layers.Dense(units=hp_filters4, activation='relu'))
    model.add(tf.keras.layers.Dropout(hp_dropout4))

    model.add(tf.keras.layers.Dense(units=4, activation='softmax')) 


    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

    return model



In [ ]:
# Hyperband determines the number of models to train in a bracket by 
# computing 1 + log factor(max_epochs) and rounding it up to the nearest integer.

tuner = kt.Hyperband(model_builder,
                    objective='val_accuracy',
                    max_epochs=70,
                    factor=3,
                    directory='../data',
                    project_name='CNN_test_initializer_v1') #always change from run to run, otherwise triggered exit

                    


In [ ]:
#tuner.search(img_train, label_train, epochs=70, validation_split=0.2, callbacks=[stop_early]) #for original train data
tuner.search(img_os, label_os, epochs=10, validation_split=0.2, callbacks=[stop_early], verbose = 0) #for oversampled ata 

# get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

/Users/carolinmundinger/.pyenv/versions/3.11.3/lib/python3.11/site-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-08-11 15:16:26.541785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-11 15:17:13.750276: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-11 15:19:45.373469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-11 15:20:39.422292: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-11 15:23:28.043294: I tensorflow/core

KeyboardInterrupt: 

## Modelling Ensemble Model

We will create en ensemble model, that combines different models for different parts of the data.


### Part 1: Convolutional neural network for the spectral data

In [ ]:
# Create model structure
input_spec = tf.keras.layers.Input(shape=(35,35,4))
scale_spec = tf.keras.layers.Lambda(lambda x: x/((2**16)-1), input_shape=(35,35,4))(input_spec)

conv1_spec = tf.keras.layers.Conv2D(filters=32, kernel_size=[3,3], activation='relu')(scale_spec)
pool1_spec = tf.keras.layers.MaxPooling2D(pool_size=[3,3])(conv1_spec)
drop1_spec = tf.keras.layers.Dropout(0.2)(pool1_spec)

conv2_spec = tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3], activation='relu')(drop1_spec)
pool2_spec = tf.keras.layers.MaxPooling2D(pool_size=[3,3])(conv2_spec)
drop2_spec = tf.keras.layers.Dropout(0.4)(pool2_spec)

conv3_spec = tf.keras.layers.Conv2D(filters=128, kernel_size=[3,3], activation='relu')(drop2_spec)
drop3_spec = tf.keras.layers.Dropout(0.2)(conv3_spec)

flat1_spec = tf.keras.layers.Flatten()(drop3_spec)

# dens1_spec = tf.keras.layers.Dense(units=32, activation='relu')(flat1_spec)
# drop4_spec = tf.keras.layers.Dropout(0.25)(dens1_spec)

# output_spec = tf.keras.layers.Dense(units=3, activation='softmax')(drop4_spec)

model_spec = tf.keras.Model(input_spec, flat1_spec, name="model_spec")

model_spec.summary()

Model: "model_spec"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 35, 35, 4)]       0         
                                                                 
 lambda (Lambda)             (None, 35, 35, 4)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 33, 33, 32)        1184      
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout_4 (Dropout)         (None, 11, 11, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 9, 9, 64)          18496     
                                                        

### Part 2: Convolutional neural network for the vegetation height data

In [ ]:
# Define  the convolutional neural network for the vegetation height data

input_height = tf.keras.layers.Input(shape=(11,11,1))
scale_height = tf.keras.layers.Lambda(lambda x: ((x-min_height)/(max_height-min_height)), input_shape=(11,11,1))(input_height)

conv1_height = tf.keras.layers.Conv2D(filters=32, kernel_size=[3,3], activation='relu')(scale_height)
pool1_height = tf.keras.layers.MaxPooling2D(pool_size=[3,3])(conv1_height)
drop1_height = tf.keras.layers.Dropout(0.4)(pool1_height)

conv2_height = tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3], activation='relu')(drop1_height)
drop2_height = tf.keras.layers.Dropout(0.2)(conv2_height)

flat1_height = tf.keras.layers.Flatten()(drop2_height)

# dens1_height = tf.keras.layers.Dense(units=32, activation='relu')(flat1_height)
# drop4_height = tf.keras.layers.Dropout(0.25)(dens1_height)

# output_height = tf.keras.layers.Dense(units=3, activation='softmax')(drop4_height)

model_height = tf.keras.Model(input_height, flat1_height, name="model_height")

model_height.summary()

Model: "model_height"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 11, 11, 1)]       0         
                                                                 
 lambda_1 (Lambda)           (None, 11, 11, 1)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 9, 9, 32)          320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 3, 3, 32)          0         
 g2D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 3, 3, 32)          0         
                                                                 
 conv2d_7 (Conv2D)           (None, 1, 1, 64)          18496     
                                                      

### Combine both models

In [ ]:
# Combine the different convolutional neural networks to one inference

# Concatenate both models from before
cross_speight = tf.keras.layers.concatenate([flat1_spec, flat1_height]) 

# Add more layers:
dens1_speight = tf.keras.layers.Dense(units=32, activation='relu')(cross_speight)
drop1_speight = tf.keras.layers.Dropout(.4)(dens1_speight)

dens2_speight = tf.keras.layers.Dense(units=16, activation='relu')(drop1_speight)
drop2_speight = tf.keras.layers.Dropout(.2)(dens2_speight)

# For the last output layer:
# units set to 4 because we have 4 classes
# softmax is commonly used for multi-class single-label classificiation
output_speight = tf.keras.layers.Dense(units=4, activation='softmax')(drop2_speight)

model_speight = tf.keras.models.Model(inputs=[input_spec, input_height], outputs=output_speight, name="model_speight")

model_speight.summary()


Model: "model_speight"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 35, 35, 4)]          0         []                            
                                                                                                  
 lambda (Lambda)             (None, 35, 35, 4)            0         ['input_2[0][0]']             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 33, 33, 32)           1184      ['lambda[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)           0         ['conv2d_3[0][0]']            
 D)                                                                                   

In [ ]:
model_speight.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.1),
              metrics=['categorical_accuracy'])

history_speight = model_speight.fit(x=[img_os, height_os], y=label_os_ohe, epochs=25, validation_data=([img_test, height_test], label_test_ohe))

In [ ]:
# Plot the loss function
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(history_speight.history['loss'], 'r', label='train')
ax.plot(history_speight.history['val_loss'], 'b' ,label='val')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Loss', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20)

# Plot the accuracy
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(history_speight.history["categorical_accuracy"], 'r', label='train')
ax.plot(history_speight.history['val_categorical_accuracy'], 'b' ,label='val')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Accuracy', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20)

## Results of the ensemble model

In [ ]:
# Evaluate the performance of the model
model_speight.evaluate(x=[img_validation, height_validation], y=label_validation_ohe)

### Create prediction for the validation dataset and evaluate performance

In [ ]:
# Predict from fitted model
predictions = model_speight.predict([img_validation, height_validation], verbose=0)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate class percentages
class_percentages = np.unique(label_validation, return_counts=True)[1]/len(label_validation) * 100

# Calculate predicted class percentages
pred_class_percentages = np.unique(predicted_labels, return_counts=True)[1]/len(predicted_labels) * 100

# Define class names
CLASS_NAMES = ['Other', 'Rotbuche','Ahorn','Birke'] # Stimmt die Reihenfolge, Alex?



Let´s compare the actural proportions of each class in the original data versus the predicted proportions

In [ ]:
print('Actual proportions')
# Look at class percentages
for class_name, percentage in zip(CLASS_NAMES, class_percentages):
    print(f'{class_name}: {percentage:.2f}%')

print('\n')
print('Predicted proportions')
# Look at predicted class percentages
for class_name, percentage in zip(CLASS_NAMES, pred_class_percentages):
    print(f'{class_name}: {percentage:.2f}%')

### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Create matrix
confusion = confusion_matrix(np.reshape(label_validation, (5,-1))[1,:], np.reshape(predicted_labels, (5,-1))[1,:])

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues",
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Classification report
report = classification_report(label_validation, predicted_labels, target_names=CLASS_NAMES)
print(report)

## How well does the trained model work on other datasets?

Here, we test how well our fitted model (that was trained on the data from 2022) can be transferred to another data set.
As an example, we test our fitted model for the data from 2018.

In [ ]:
# Define the path to the data file
data_path = '../images/box_images/baseline/model_images.npz'

# Load the image array containing all box images in a 4D array and their
# labels
with np.load(data_path, allow_pickle=True) as data:
    img_array = data["img_array"]
    labels = data["labels"]

# Extract only none nan values from the year 2018 
img_array = img_array[:,0:1701,:,:,:]
img_array = np.reshape(img_array[[2,2,2,2,2],:,:,:,:], (-1,35,35,4)) ## 2 refers to 2018
height_array = np.reshape(height_array, (-1,11,11))
label_array = np.reshape(labels[:,0:1701], (-1))

# As tensorflow prefers integers compared to strings as target values, we 
# transform our labels array. As a first step, create a new array that
# looks exactly like labels but is filled with zeros
labels_int = np.zeros_like(label_array)

# Loop through each label
for i in range(len(label_array)):
    
    # Assign a 1 if it is a Rotbuche
    if label_array[i] == 'Rotbuche':
        labels_int[i] = 1

    # Assign a 2 if it is a Berg-Ahorn    
    elif label_array[i] == 'Berg-Ahorn':
        labels_int[i] = 2

    # Assign a 3 if it is a Haenge-Birke    
    elif label_array[i] == 'Haenge-Birke':
        labels_int[i] = 3
    
# As the array is still of type "obj", we need to change it to "int"
labels_int = labels_int.astype('int')

In [ ]:
np.unique(labels_int, return_counts=True)

In [ ]:
# Predict from fitted model
predictions = model_speight.predict([img_array, height_array], verbose=0)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate class percentages
class_percentages = np.unique(labels_int, return_counts=True)[1]/len(labels_int) * 100

# Calculate predicted class percentages
pred_class_percentages = np.unique(predicted_labels, return_counts=True)[1]/len(predicted_labels) * 100

# Define class names
CLASS_NAMES = ['Other', 'Rotbuche','Ahorn','Birke'] # Stimmt die Reihenfolge, Alex?

print('Actual proportions')
# Look at class percentages
for class_name, percentage in zip(CLASS_NAMES, class_percentages):
    print(f'{class_name}: {percentage:.2f}%')

print('\n')
print('Predicted proportions')
# Look at predicted class percentages
for class_name, percentage in zip(CLASS_NAMES, pred_class_percentages):
    print(f'{class_name}: {percentage:.2f}%')

In [ ]:
np.mean(labels_int == predicted_labels)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Create matrix
confusion = confusion_matrix(np.reshape(labels_int, (5,-1))[1,:], np.reshape(predicted_labels, (5,-1))[1,:])

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues",
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()